Dolomite-Baseline: for fine-tuning Falcon-7b over A100-40GB

* repo setup
* checkpoint download - tiiuae/falcon-7b
* inference test
* finetune: reproduce stable baseline 50k_iters  
* finetune: search new baseline with auto-checkpoints and longer training



In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


Sun Aug 13 22:40:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Phase 1. Setup repo, install dependencies, download model weights

In [3]:
!git clone https://github.com/Lightning-AI/lit-gpt
!git clone https://github.com/alicata/llm-dolomite-base.git
!cp llm-dolomite-base/lora_* lit-gpt/finetune

%cd lit-gpt

# Main repo with CUDA and flash attention
!pip install --index-url https://download.pytorch.org/whl/nightly/cu118 --pre 'torch>=2.1.0dev' -q
# Remove audio dependency - integration bug
!pip uninstall -y torchaudio
# install the dependencies
!pip install -r requirements.txt
!pip install huggingface_hub

# download the original Falcon-7B weights and convert into Lit-GPT compatible model format
!python scripts/download.py --repo_id tiiuae/falcon-7b
!python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/tiiuae/falcon-7b

Cloning into 'lit-gpt'...
remote: Enumerating objects: 3254, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 3254 (delta 151), reused 184 (delta 93), pack-reused 2989
Receiving objects: 100% (3254/3254), 1005.33 KiB | 3.84 MiB/s, done.
Resolving deltas: 100% (2183/2183), done.
Cloning into 'llm-dolomite-base'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 89 (delta 40), reused 4 (delta 1), pack-reused 0
Receiving objects: 100% (89/89), 15.59 MiB | 5.52 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/lit-gpt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 843.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 MB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

Phase 2. Test Inference.

In [4]:
# run inference
!python generate/base.py \
        --prompt "Hello, my name is" \
        --checkpoint_dir checkpoints/tiiuae/falcon-7b \
        --quantize bnb.int8

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.utils:Representation` has been removed. We are importing from `pydantic.v1.utils:Representation` instead.See the migration guide for more details: https://docs.pydantic.dev/latest/migration/
  warnings.warn(
Loading model 'checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 512, 'padded_vocab_size': 65024, 'n_layer': 32, 'n_head': 71, 'n_embd': 4544, 'rotary_percentage': 1.0, 'parallel_residual': True, 'bias': False, 'n_query_groups': 1, 'shared_attention_norm': True, '_norm_class': 'LayerNorm', 'norm_eps': 1e-05, '_mlp_class': 'GptNeoxMLP', 'intermediate_size': 18176, 'condense_ratio': 1}
Time to instantiate model: 1.69 seconds.
Time to load the model weights: 13.94 seconds.
Global seed set to 1234
Hello, my name is Jack.
Some people think that dogs are like people. They see them as loyal, loving, in

Phase 3. Generate Synthetic Dataset





In [5]:
#python scripts/prepare_alpaca.py --checkpoint_dir checkpoints/stabilityai/stablelm-base-alpha-3b
!python scripts/prepare_alpaca.py --checkpoint_dir checkpoints/tiiuae/falcon-7b
#!python scripts/prepare_alpaca.py

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.utils:Representation` has been removed. We are importing from `pydantic.v1.utils:Representation` instead.See the migration guide for more details: https://docs.pydantic.dev/latest/migration/
  warnings.warn(
Loading data file...
Loading tokenizer...
train has 49,759 samples
test has 2,000 samples
Processing train split ...
100% 49759/49759 [00:38<00:00, 1290.13it/s]
Processing test split ...
100% 2000/2000 [00:01<00:00, 1229.47it/s]


# Phase 4. Finetuning Overview

Choose between reproducing and searching new baselines
* 4.A stable baseline reproduction for falcon-7b_50k_iters test (A100-40GB)
* 4.B for training longer experiments on Lambda or ColabPro

# Phase 4.A Finetune to Reproduce Baseline
* use patched repo (lora_compact.py)
* reproduce baseline falcon-7b_50k_iters


In [6]:
#!python finetune/adapter_v2_small.py --checkpoint_dir checkpoints/tiiuae/falcon-7b
!python finetune/lora_compact.py  --precision bf16-true --checkpoint_dir checkpoints/tiiuae/falcon-7b

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.utils:Representation` has been removed. We are importing from `pydantic.v1.utils:Representation` instead.See the migration guide for more details: https://docs.pydantic.dev/latest/migration/
  warnings.warn(
Missing logger folder: out/lora/alpaca
{'eval_interval': 50, 'save_interval': 100, 'eval_iters': 100, 'log_interval': 100, 'devices': 1, 'override_max_seq_length': 100, 'learning_rate': 0.0003, 'batch_size': 128, 'micro_batch_size': 2, 'gradient_accumulation_iters': 64, 'max_iters': 100000, 'weight_decay': 0.01, 'lora_r': 8, 'lora_alpha': 16, 'lora_dropout': 0.05, 'lora_query': True, 'lora_key': False, 'lora_value': True, 'lora_projection': False, 'lora_mlp': False, 'lora_head': False, 'warmup_steps': 100}
Global seed set to 1337
Loading model 'checkpoints/tiiuae/falcon-7b/lit_model.pth' with {'org': 'tiiuae', 'name': 'falcon-7b', 'block_size': 2048, 'vocab_size': 50254, 'padding_multiple': 5

# Phase 4.B Finetune New Baseline
* modified lora.py for GPU memory and auto-resume checkpoints
* integrated into this notebook
* modify experiments in expr {} dictionary below
* checkpoints dir and output dirs are implicit for lora-falcon-alpaca in the code default inputs


In [ ]:
!python finetune/lora_baselines.py  --precision bf16-true --checkpoint_dir checkpoints/tiiuae/falcon-7b --max_iters 10000

# Store result in Google Drive


In [7]:
!mkdir -p /contents/drive/MyDrive/Colab\ Notebooks/ViT/falcon-7b/checkpoints/tiiuae/falcon-7b
!cp checkpoints/tiiuae/falcon-7b/*.json /contents/drive/MyDrive/Colab\ Notebooks/ViT/falcon-7b/checkpoints/*.json

!mkdir -p /contents/drive/MyDrive/Colab\ Notebooks/ViT/falcon-7b/data
!cp -r data /contents/drive/MyDrive/Colab\ Notebooks/ViT/falcon-7b/data

!mkdir -p /contents/drive/MyDrive/Colab\ Notebooks/ViT/falcon-7b/out/lora/alpaca
!cp out/lora/alpaca/iter-09*.pth /contents/drive/MyDrive/Colab\ Notebooks/ViT/falcon-7b/out/lora/alpaca




cp: target '/contents/drive/MyDrive/Colab Notebooks/ViT/falcon-7b/checkpoints/*.json' is not a directory


# Reference lora_baselins.py

In [ ]:
# experimental setup to search new baseline
expr = {
    'max_iters'        : 50000,        # ~2h CP
    'resume'           : True,         # auto resume from out_dir intermediate checkpoints (i.e. out/lora/alpaca/*.)
    'precision'        : 'bf16-true',  # reduce GPU peak memory < 24GB
    'micro_batch_size' : 2,            # reduce GPU peak memory < 24GB
}


In [ ]:
import os
import sys
import time
from pathlib import Path
from typing import Dict, List, Optional, Tuple



import lightning as L
import torch
from lightning.fabric.strategies import FSDPStrategy, XLAStrategy

# support running without installing as a package
wd = Path(__file__).parent.parent.resolve()
sys.path.append(str(wd))

from generate.base import generate
from lit_gpt.lora import GPT, Block, Config, lora_filter, mark_only_lora_as_trainable
from lit_gpt.speed_monitor import SpeedMonitorFabric as SpeedMonitor
from lit_gpt.speed_monitor import estimate_flops, measure_flops
from lit_gpt.tokenizer import Tokenizer
from lit_gpt.utils import check_valid_checkpoint_dir, chunked_cross_entropy, lazy_load, num_parameters, step_csv_logger
from scripts.prepare_alpaca import generate_prompt

eval_interval = 50
save_interval = 100
eval_iters = 100
log_interval = 100
devices = 1
# change this value to force a maximum sequence length
override_max_seq_length = 100

# Hyperparameters
learning_rate = 3e-4
batch_size = 128
micro_batch_size = expr['micro_batch_size']
gradient_accumulation_iters = batch_size // micro_batch_size
assert gradient_accumulation_iters > 0
max_iters = expr['max_iters']  # train dataset size
weight_decay = 0.01
lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
lora_query = True
lora_key = False
lora_value = True
lora_projection = False
lora_mlp = False
lora_head = False
warmup_steps = 100

hparams = {k: v for k, v in locals().items() if isinstance(v, (int, float, str)) and not k.startswith("_")}


def setup(
    data_dir: Path = Path("data/alpaca"),
    checkpoint_dir: Path = Path("checkpoints/tiiuae/falcon-7b"),
    out_dir: Path = Path("out/lora/alpaca"),
    precision: Optional[str] = None,
    tpu: bool = False,
    resume: Union[bool, Path] = False,
):
    precision = expr['precision']
    resume = expr['resume']

    if precision is None:
        precision = "32-true" if tpu else "bf16-mixed"
    fabric_devices = devices
    if fabric_devices > 1:
        if tpu:
            # For multi-host TPU training, the device count for Fabric is limited to the count on a single host.
            fabric_devices = "auto"
            strategy = XLAStrategy(sync_module_states=False)
        else:
            strategy = FSDPStrategy(
                auto_wrap_policy={Block},
                activation_checkpointing_policy={Block},
                state_dict_type="full",
                limit_all_gathers=True,
            )
    else:
        strategy = "auto"

    logger = step_csv_logger(out_dir.parent, out_dir.name, flush_logs_every_n_steps=log_interval)
    fabric = L.Fabric(devices=fabric_devices, strategy=strategy, precision=precision, loggers=logger)
    fabric.print(hparams)
    fabric.launch(main, data_dir, checkpoint_dir, out_dir, resume)


def main(fabric: L.Fabric, data_dir: Path, checkpoint_dir: Path, out_dir: Path, resume: bool):
    check_valid_checkpoint_dir(checkpoint_dir)

    speed_monitor = SpeedMonitor(fabric, window_size=50, time_unit="seconds")

    fabric.seed_everything(1337)  # same seed for every process to init model (FSDP)

    if fabric.global_rank == 0:
        os.makedirs(out_dir, exist_ok=True)

    train_data = torch.load(data_dir / "train.pt")
    val_data = torch.load(data_dir / "test.pt")

    if not any((lora_query, lora_key, lora_value, lora_projection, lora_mlp, lora_head)):
        fabric.print("Warning: all LoRA layers are disabled!")
    config = Config.from_name(
        name=checkpoint_dir.name,
        r=lora_r,
        alpha=lora_alpha,
        dropout=lora_dropout,
        to_query=lora_query,
        to_key=lora_key,
        to_value=lora_value,
        to_projection=lora_projection,
        to_mlp=lora_mlp,
        to_head=lora_head,
    )
    checkpoint_path = checkpoint_dir / "lit_model.pth"
    fabric.print(f"Loading model {str(checkpoint_path)!r} with {config.__dict__}")
    with fabric.init_module(empty_init=False):
        model = GPT(config)
        model.apply(model._init_weights)  # for the LoRA weights
    with lazy_load(checkpoint_path) as checkpoint:
        # strict=False because missing keys due to LoRA weights not contained in state dict
        model.load_state_dict(checkpoint, strict=False)

    mark_only_lora_as_trainable(model)

    fabric.print(f"Number of trainable parameters: {num_parameters(model, requires_grad=True):,}")
    fabric.print(f"Number of non trainable parameters: {num_parameters(model, requires_grad=False):,}")
    trainable_params = [p for p in model.parameters() if p.requires_grad]

    optimizer = torch.optim.AdamW(trainable_params, lr=learning_rate, weight_decay=weight_decay)
    model, optimizer = fabric.setup(model, optimizer)

    fabric.seed_everything(1337 + fabric.global_rank)

    state = {"model": model, "optimizer": optimizer, "hparams": hparams, "iter_num": 0, "step_count": 0}
    if resume is True:
        resume = sorted(out_dir.glob("*.pth"))[-1]
    if resume:
        fabric.print(f"Resuming training from {resume}")
        fabric.load(resume, state)

    train_time = time.time()
    train(fabric, state, train_data, val_data, checkpoint_dir, out_dir, speed_monitor)
    #train(fabric, model, optimizer, train_data, val_data, checkpoint_dir, out_dir, speed_monitor)
    fabric.print(f"Training time: {(time.time()-train_time):.2f}s")

    # Save the final LoRA checkpoint at the end of training
    save_path = out_dir / "lit_model_lora_finetuned.pth"
    save_lora_checkpoint(fabric, model, save_path)


def train(
    fabric: L.Fabric,
    state: Dict,
    # model: GPT,
    # optimizer: torch.optim.Optimizer,
    train_data: List[Dict],
    val_data: List[Dict],
    checkpoint_dir: Path,
    out_dir: Path,
    speed_monitor: SpeedMonitor,
) -> None:
    tokenizer = Tokenizer(checkpoint_dir)
    max_seq_length, longest_seq_length, longest_seq_ix = get_max_seq_length(train_data)

    # loaded
    model = state["model"]
    optimizer = state["optimizer"]

    validate(fabric, model, val_data, tokenizer, longest_seq_length)  # sanity check

    with torch.device("meta"):
        meta_model = GPT(model.config)
        # estimated is too much of an optimistic estimate, left just for reference
        estimated_flops = estimate_flops(meta_model) * micro_batch_size
        fabric.print(f"Estimated TFLOPs: {estimated_flops * fabric.world_size / 1e12:.2f}")
        x = torch.randint(0, 1, (micro_batch_size, model.config.block_size))
        measured_flops = measure_flops(meta_model, x)
        fabric.print(f"Measured TFLOPs: {measured_flops * fabric.world_size / 1e12:.2f}")
        del meta_model, x

    step_count = 0
    total_lengths = 0
    total_t0 = time.time()

    if fabric.device.type == "xla":
        import torch_xla.core.xla_model as xm

        xm.mark_step()
    for iter_num in range(max_iters):
        if step_count <= warmup_steps:
            # linear warmup
            lr = learning_rate * step_count / warmup_steps
            for param_group in optimizer.param_groups:
                param_group["lr"] = lr

        iter_t0 = time.time()

        input_ids, targets = get_batch(
            fabric, train_data, longest_seq_length, longest_seq_ix if iter_num == 0 else None
        )

        is_accumulating = (iter_num + 1) % gradient_accumulation_iters != 0
        with fabric.no_backward_sync(model, enabled=is_accumulating):
            logits = model(input_ids, max_seq_length=max_seq_length, lm_head_chunk_size=128)
            # shift the targets such that output n predicts token n+1
            logits[-1] = logits[-1][..., :-1, :]
            loss = chunked_cross_entropy(logits, targets[..., 1:])
            fabric.backward(loss / gradient_accumulation_iters)

        if not is_accumulating:
            optimizer.step()
            optimizer.zero_grad()
            step_count += 1
        elif fabric.device.type == "xla":
            xm.mark_step()

        t1 = time.time()
        total_lengths += input_ids.size(1)
        speed_monitor.on_train_batch_end(
            (iter_num + 1) * micro_batch_size,
            t1 - total_t0,
            # this assumes that device FLOPs are the same and that all devices have the same batch size
            fabric.world_size,
            flops_per_batch=measured_flops,
            lengths=total_lengths,
        )
        if iter_num % log_interval == 0:
            elapsed = t1 - total_t0
            fabric.print(
                    f"iter {iter_num} step {step_count}: loss {loss.item():.4f},  time:{elapsed/60:.1f}mins  iter time:"
                f" {(t1 - iter_t0) * 1000:.2f}ms{' (optimizer.step)' if not is_accumulating else ''}"
            )

        if not is_accumulating and step_count % eval_interval == 0:
            t0 = time.time()
            val_loss = validate(fabric, model, val_data, tokenizer, longest_seq_length)
            t1 = time.time() - t0
            speed_monitor.eval_end(t1)
            fabric.print(f"step {iter_num}: val loss {val_loss:.4f}, val time: {t1 * 1000:.2f}ms")
            fabric.barrier()
        if not is_accumulating and step_count % save_interval == 0:
            checkpoint_path = out_dir / f"iter-{iter_num:06d}-ckpt.pth"
            save_lora_checkpoint(fabric, model, checkpoint_path)


@torch.no_grad()
def validate(
    fabric: L.Fabric, model: GPT, val_data: List[Dict], tokenizer: Tokenizer, longest_seq_length: int
) -> torch.Tensor:
    fabric.print("Validating ...")
    model.eval()
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        input_ids, targets = get_batch(fabric, val_data, longest_seq_length)
        logits = model(input_ids)
        loss = chunked_cross_entropy(logits, targets, chunk_size=0)
        losses[k] = loss.item()
    val_loss = losses.mean()

    # produce an example:
    instruction = "Recommend a movie for me to watch during the weekend and explain the reason."
    fabric.print(instruction)
    sample = {"instruction": instruction, "input": ""}
    prompt = generate_prompt(sample)
    encoded = tokenizer.encode(prompt, device=fabric.device)
    max_returned_tokens = len(encoded) + 100
    output = generate(
        model, idx=encoded, max_returned_tokens=max_returned_tokens, max_seq_length=max_returned_tokens, temperature=0.8
    )
    output = tokenizer.decode(output)
    fabric.print(output)

    model.reset_cache()

    model.train()
    return val_loss.item()


def get_batch(
    fabric: L.Fabric, data: List[Dict], longest_seq_length: int, longest_seq_ix: Optional[int] = None
) -> Tuple[torch.Tensor, torch.Tensor]:
    ix = torch.randint(len(data), (micro_batch_size,))
    if longest_seq_ix is not None:
        # force the longest sample at the beginning so potential OOMs happen right away
        ix[0] = longest_seq_ix

    input_ids = [data[i]["input_ids"].type(torch.int64) for i in ix]
    labels = [data[i]["labels"].type(torch.int64) for i in ix]

    # it's better to pad to a fixed seq length with XLA to avoid recompilation
    max_len = max(len(s) for s in input_ids) if fabric.device.type != "xla" else longest_seq_length

    def pad_right(x, pad_id):
        # pad right based on the longest sequence
        n = max_len - len(x)
        return torch.cat((x, torch.full((n,), pad_id, dtype=x.dtype)))

    x = torch.stack([pad_right(x, pad_id=0) for x in input_ids])
    y = torch.stack([pad_right(x, pad_id=-1) for x in labels])

    if fabric.device.type == "cuda" and x.device.type == "cpu":
        x, y = fabric.to_device((x.pin_memory(), y.pin_memory()))
    else:
        x, y = fabric.to_device((x, y))
    return x, y


def get_max_seq_length(data: List[Dict]) -> Tuple[int, int, int]:
    # find out the minimum max_seq_length required during fine-tuning (saves memory!)
    lengths = [len(d["input_ids"]) for d in data]
    max_seq_length = max(lengths)
    longest_seq_ix = lengths.index(max_seq_length)
    # support easy override at the top of the file
    return (
        override_max_seq_length if isinstance(override_max_seq_length, int) else max_seq_length,
        max_seq_length,
        longest_seq_ix,
    )


def save_lora_checkpoint(fabric, model, file_path: Path):
    fabric.print(f"Saving LoRA weights to {str(file_path)!r}")
    fabric.save(file_path, {"model": model}, filter={"model": lora_filter})


if __name__ == "__main__":
    # Uncomment this line if you see an error: "Expected is_sm80 to be true, but got false"
    # torch.backends.cuda.enable_flash_sdp(False)
    torch.set_float32_matmul_precision("high")

    from jsonargparse import CLI

    CLI(setup)


# Appendix - Fixes, and Issue Resolutions

## Unsolved
* automate pytorch audio uninstall as is interactive (prompts for Y)
* auto save intermediate checkpoints to GDrive automatically to allow shutting down run-time anytime
* test moving checkpoints from/to GDrive to/from Lambda + ColabPro

## Solved
* uninstall pytorch audio - nigtly version of pytorch with cuda conflicts with pytorch audio and cannot integrate with lit-gpt
* CUDA OOM - reduce micro_batch_size to 2, use bfp16 to avoid pytorch memory peak spikes and keep < 24GB VRAM
* max_iters = 50k takes 2 GPU/hrs training, and insufficient for natural-english-like baseline
* patch lora.py with above workarounds - rename lora_compact.py
* integrate notebook for GPU server - hard to experiment and fine new baseline, make easy to customize fine-turning for A100-40GB on GPU server
* auto-resume: for cost reduction of fine-turning or mixing resuming checkpoints on different GPU servers

